In [6]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict, CTCLabelConverter
import pandas as pd

In [7]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [8]:
opt = get_config("config_files/en_filtered_config.yaml")
opt

{'number': '0123456789',
 'symbol': '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ €',
 'lang_char': 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz',
 'experiment_name': 'en_filtered',
 'train_data': 'all_data',
 'valid_data': 'all_data/eval_img',
 'manualSeed': 1111,
 'workers': 6,
 'batch_size': 32,
 'num_iter': 300000,
 'valInterval': 20000,
 'saved_model': './weights/english.pth',
 'FT': False,
 'optim': False,
 'lr': 1.0,
 'beta1': 0.9,
 'rho': 0.95,
 'eps': 1e-08,
 'grad_clip': 5,
 'select_data': 'train_img',
 'batch_ratio': '1',
 'total_data_usage_ratio': 1.0,
 'batch_max_length': 34,
 'imgH': 64,
 'imgW': 600,
 'rgb': False,
 'contrast_adjust': 0.0,
 'sensitive': True,
 'PAD': True,
 'data_filtering_off': False,
 'Transformation': 'None',
 'FeatureExtraction': 'MobileNet',
 'SequenceModeling': 'BiLSTM',
 'Prediction': 'CTC',
 'num_fiducial': 20,
 'input_channel': 1,
 'output_channel': 256,
 'hidden_size': 256,
 'decode': 'greedy',
 'new_prediction': False,
 'freeze_FeatureFxtractio

In [9]:
converter = CTCLabelConverter(opt.character)
opt.num_class = len(converter.character)

In [10]:
from model import Model
model = Model(opt)

No Transformation module specified


In [11]:
model.eval()

Model(
  (FeatureExtraction): MobileNetV3(
    (conv1): ConvBNLayer(
      (conv): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Activation(
        (act): Hswish()
      )
    )
    (blocks): Sequential(
      (0): ResidualUnit(
        (expand_conv): ConvBNLayer(
          (conv): Conv2d(8, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): Activation(
            (act): ReLU(inplace=True)
          )
        )
        (bottleneck_conv): ConvBNLayer(
          (conv): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), groups=8, bias=False)
          (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): Activation(
            (act): ReLU(inplace=True)
          )
        )
        (m

In [13]:
model.FeatureExtraction.blocks

Sequential(
  (0): ResidualUnit(
    (expand_conv): ConvBNLayer(
      (conv): Conv2d(8, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Activation(
        (act): ReLU(inplace=True)
      )
    )
    (bottleneck_conv): ConvBNLayer(
      (conv): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), groups=8, bias=False)
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): Activation(
        (act): ReLU(inplace=True)
      )
    )
    (mid_se): SEModule(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (conv1): Conv2d(8, 2, kernel_size=(1, 1), stride=(1, 1))
      (relu1): Activation(
        (act): ReLU(inplace=True)
      )
      (conv2): Conv2d(2, 8, kernel_size=(1, 1), stride=(1, 1))
      (hard_sigmoid): Activation(
        (act): Hsigmoid()
      )
    )
    (linear_conv): ConvBNLayer(
      (conv): Conv2d(